# JC Model

### Job Creator parameters

$r = instructionCount*JO.rate + bandwidthUsage*JO.rate$ : The reward the JC is willing to pay to have a job executed.

$pID = 1$ : The probability that JC will identify a job as class 1, 2, or junk.

$p_{v1}$ : probability the JC verifies answers from class 1. 

$p_{v2}$ : probability the JC verifies answers from class 2. This probability is 0 because it can just go straight to mediation. 

$vc$ : cost to JC to verify result.

$b$ : the value or benefit JC gets when a job is performed correctly.

$matchPrice$ : the amount JC pays to Solver for finding a suitable RP. 

$mediatorPrice$ : The amount JC pays to mediator for being available. 

$p_1$ : probability that job returns answer in class $a1$

$p_2 = 1-p_1$ : probability that job returns answer in class $a2$

$deposit = price\times PR\times n$ : the amount the Job Creator must post as a security deposit in order to get matched. It multiplied by the $n$, which as a reminder is the number of times the mediator will replicate a job. 

In [ ]:
def JCU(n,PR,NDPR,  b,r,p1,pID,pv1,vc1,pv2,vc2,  pq):
    n,PR,NDPR, pID=1,p1=p1,b=benefit,r=reward,vc1=vc1, pq=pq
    
    honest = pID*pv1*p1*pq * (b-r-vc1)
    lazy   = pID*(1-pv1)*p1*pq * (b-r)
    lucky  = (1-pID)*
    stupid
    normal = honest + lazy + lucky + stupid
    
    NDGain =
    NDFine = 
    ND = NDGain + NDFine
    
    RPfraudWin
    RPfraudLose
    RPfraudScared
    RPfraud =  RPfraudWin + RPfraudLose + RPfraudScared
    
    
    return normal + ND + RPfraud
    
        p2 = 1-p1
    deposit = price * M_PR * M_n
    NDdeposit = price * M_NDPR
    
    noVerifyCost = -(1-p)*price
    
    verifyCost   = -p*jcj # chance to check job (p) and recompute (cj)
    A1payout     =  p*q*p1*(benefit-price) # RP correctly (q) returns answer 1 (p1) and pay up to max_price (-price).
    compensation =  p*q*p2*(p1**M_n)*(benefit + price + JCgasPrice) # receives gas price because it had to spend some to request mediation.
    NDfine       = -p*q*p2*(1-(p1**M_n))*NDdeposit #RP returns answer 2 correctly or responds incorrectly (p2+(1-q)) and send to mediator who finds out nonDeterminism (1-p1^n), pay fine and reward (f2+r)
    
    mediationCost= -p*(1-q)*JCgasPrice
    RPfraudC     =  p*(1-q)*(p1**M_n)*(price + JCgasPrice) 
    RPfraudL     = -p*(1-q)*(1-p1**M_n)*NDdeposit# compute incorrectly (1-q) get sent to mediation, finds nondeterminism (1-p1^n) get compensation.
    RPfraud1     =  RPfraudC + RPfraudL
    
    RPfraud2     = -p*(1-q)*price # JC doesn't want to risk getting caught, just pay RP. 
    
    JCnonDet = verifyCost + A1payout + compensation + NDfine + max(RPfraud1,RPfraud2) + mediationCost
    return JCnonDet

In [1]:
def plotUtil(n, PR, NDPR,  r,benefit,reward,vc1,   pq):
    deposit = r * PR * n
    NDdeposit = r * NDPR
    
    plt.subplot(1,1,1)
    plt.grid(visible=True)
    x,step = np.linspace(0,1,51,retstep=True)
    print("step size: %s" %step)
    
    yJND = list(map(lambda p1: JCU(n,PR,NDPR, pID=1,p1=p1,b=benefit,r=reward,vc1=vc1, pq=pq),x))
    yJ   = list(map(lambda p1: JCU(n,PR,NDPR, pID=1,p1=1,b=benefit,r=reward,vc1=vc1, pq=pq),x)
                
#     yRND = list(map(lambda p1: RPNDU(M_n,M_PR,M_NDPR, p1,price,p=p, q=q,roi=roi,I=Insurance),x))
#     yRNDD = list(map(lambda p1: RPNDUD(M_n,M_PR,M_NDPR,p1,price,p=p,rcj=rcj,q=q,roi=roi,I=Insurance),x))
    plt.plot(x,yJ, label="JU")
    plt.plot(x,yJND, label="JUND")
#     plt.plot(x,yRND, label="RUND")
#     plt.plot(x,yRNDD, label="RUNDD")
    plt.xlabel('P(a1)')
    plt.ylabel('Util') 
    
    plt.legend()
    plt.show

SyntaxError: invalid syntax (<ipython-input-1-75b56c146354>, line 15)